In [1]:
import h5py as h5
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import sys
sys.path.append(os.path.expanduser("~/TMPredictor/survival_tm/auton-survival"))
from auton_survival.preprocessing import Scaler
import optuna
from sklearn.model_selection import ParameterGrid
#sys.path.append('/projects/EKOLEMEN/survival_tm/train_models/auton-survival')
sys.path.append(os.path.expanduser("~/TMPredictor/survival_tm/auton-survival"))
from sklearn.model_selection import ParameterGrid
from auton_survival.estimators import SurvivalModel
from auton_survival.metrics import survival_regression_metric
from auton_survival.models.dsm import DeepSurvivalMachines
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

/home/hf8585/.conda/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
shots = np.load('/projects/EKOLEMEN/survival_tm/shots.npy')
tm_shots = np.load('/projects/EKOLEMEN/survival_tm/tm_shots.npy')
st_shots = np.load('/projects/EKOLEMEN/survival_tm/st_shots.npy')

In [3]:
def load_data(data_type):
    with open(f'/projects/EKOLEMEN/survival_tm/formatted_labels/{data_type}.pkl', 'rb') as f:
        data = pickle.load(f)
    
    return data['x'], data['t'], data['e']

In [4]:
# Don't want whole dataset right now so 0.5 factor to reduce
n = len(shots)

tr_size = int(n*0.80)
vl_size = int(n*0.10)
te_size = int(n*0.10)

train_shots = shots[:tr_size]
test_shots = shots[-te_size:]
valid_shots = shots[tr_size:tr_size+vl_size]

x_train, t_train, e_train = load_data('train')
x_test,  t_test,  e_test  = load_data('test')
x_valid, t_valid, e_valid = load_data('valid')

# Get inds for time <600ms
# is this needed? Not sure.
'''inds = np.where(t_train < 600)[0]

x_train = x_train[inds]
t_train = t_train[inds]
e_train = e_train[inds]'''

tm_inds = np.where(e_train == 1)[0]
st_inds = np.where(e_train == 0)[0]
new_st_inds = np.random.choice(st_inds, size=len(tm_inds), replace=False)

x_train = np.concatenate((x_train[tm_inds], x_train[new_st_inds]), axis=0)
t_train = np.concatenate((t_train[tm_inds], t_train[new_st_inds]), axis=0)
e_train = np.concatenate((e_train[tm_inds], e_train[new_st_inds]), axis=0)

# Shuffle arrays because currently all 1s followed by all 0s
p = np.random.permutation(len(t_train))
x_train = x_train[p,:]
t_train = t_train[p]
e_train = e_train[p]


If we predict 200ms in the future, the model should see the actuator trajectories 200ms in the future. Don't show betan, just actuators. 

In [5]:
x_train_df = pd.DataFrame(x_train)
t_train_df = pd.DataFrame(t_train)
e_train_df = pd.DataFrame(e_train)

x_valid_df = pd.DataFrame(x_valid)
t_valid_df = pd.DataFrame(t_valid)
e_valid_df = pd.DataFrame(e_valid)

x_test_df = pd.DataFrame(x_test)
t_test_df = pd.DataFrame(t_test)
e_test_df = pd.DataFrame(e_test)

outcomes_valid_df = pd.DataFrame({'time': t_valid, 'event': e_valid})

outcomes_df = pd.DataFrame({'time': t_train, 'event': e_train})

#normalize
scaler = Scaler()
transformer = scaler.fit(x_train_df)
x_train_df = transformer.transform(x_train_df)
x_valid_df = transformer.transform(x_valid_df)
x_test_df = transformer.transform(x_test_df)

In [87]:
param_grid = {'k' : [3],
              'iters': [200],
              'distribution' : ['LogNormal'],
              'learning_rate' : [ 1e-3 ],
              'batch_size' : [10000],
              'layers' : [[100, 60, 175, 225, 120]]
             }

params = ParameterGrid(param_grid)
models=[]
for param in params:
    model = SurvivalModel(model='dsm', 
                      iters=param['iters'], 
                      k=param['k'], 
                      layers=param['layers'], 
                      distribution=param['distribution'],
                      learning_rate=param['learning_rate'], 
                      batch_size=param['batch_size']
                    )
    '''model = SurvivalModel(model='dsm', 
                          iters=param['iters'], 
                          k=param['k'], 
                          layers=param['layers'], 
                          distribution=param['distribution'],
                          learning_rate=param['learning_rate'], 
                          batch_size=param['batch_size']
                        )'''
    _, train_loss, val_loss = model.fit(x_train_df, outcomes_df, val_data=(x_valid_df, outcomes_valid_df))

    # Obtain survival probabilities for validation set and compute the Integrated Brier Score 
    predictions_val = model.predict_survival(x_valid_df, times)
    metric_val = survival_regression_metric('ibs', outcomes_valid_df, predictions_val, times)
    models.append([metric_val, train_loss, val_loss, model])


100%|██████████| 200/200 [3:21:10<00:00, 60.35s/it]  
/home/hf8585/TMPredictor/survival_tm/auton-survival/auton_survival/models/dsm/losses.py:406: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  t_horz = torch.tensor(t_horizon).double().to(logits.device)
/home/hf8585/TMPredictor/survival_tm/auton-survival/auton_survival/estimators.py:206: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return survival_predictions.sort_index(axis=0).interpolate().interpolate(method='bfill').T[times].values
/home/hf8585/TMPredictor/survival_tm/auton-survival/auton_survival/metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distributi

ValueError: Input estimate contains NaN.

In [78]:
with open('models.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

print(loaded_model[0][0])

In [59]:
with open('survival.pkl', 'rb') as f:
    out_survival = pickle.load(f)

#out_risk = model.predict_risk(pd.DataFrame(x_test), times)
#out_survival = model.predict_survival(pd.DataFrame(x_test), times)

In [52]:
def find_peaks_in_data(data):
    peaks = []
    for i in range(1, len(data) - 1):
        if data[i-1] < data[i] > data[i+1]:
            peaks.append(i)
    return peaks

In [41]:
prediction_times = [20, 50, 100, 200]

In [ ]:
#outcomes_train=pd.DataFrame({'time': t_train, 'event': e_train})
for i in range(len(loaded_model)):
    out_survival = loaded_model[i][0].predict_survival(x_valid_df, prediction_times)
    score = survival_regression_metric(metric='auc', outcomes=outcomes_valid_df, predictions=out_survival, times=prediction_times)
    print(score)

In [ ]:
out_survival = models[0][3].predict_survival(x_test_df, prediction_times)
peaks = find_peaks_in_data(t_test)
for i in range(0, 3):
    peak_number = 140 + i
    start_index = peaks[peak_number]
    end_index = peaks[peak_number + 1]
    times = np.arange(0, (end_index - start_index)*20, 20)
    plt.plot(times, out_survival[start_index:end_index,2], label='Survival in 100ms')
    plt.plot(times, out_survival[start_index:end_index,3], label='Survival in 200ms')

    if (e_test[start_index]==1):
        plt.title('YES TM')
    else:
        plt.title('NO TM')
    plt.xlabel('Time / ms')
    plt.ylabel('Survival probability')
    plt.ylim(0.5, 1)
    plt.legend()
    plt.show()

In [66]:
# make WTC curve

# one issue with this metric is if we never predict TMs then we're always perfect. 
def fpr_auc(model, normed_x, normed_t, normed_e, prediction_times, threshold=0.7):
    out_survival = model.predict_survival(normed_x, prediction_times)
    fprs = []
    for i, time in enumerate(prediction_times):
        survival_prediction = out_survival[:,i]
        survival_prediction = (survival_prediction < threshold).astype(int)
        false_positives = np.logical_and(survival_prediction == 1, normed_e == 0)
        true_negatives = np.logical_and(survival_prediction == 0, normed_e == 0)
        fpr = false_positives.sum() / (false_positives.sum() + true_negatives.sum())
        fprs.append(fpr)

    auc = np.trapz(fprs, prediction_times)
    return auc, fprs, prediction_times

def find_peaks_in_data(data):
    peaks = []
    for i in range(1, len(data) - 1):
        if data[i-1] < data[i] > data[i+1]:
            peaks.append(i)
    return peaks

def fnr_auc(model, normed_x, normed_t, normed_e, prediction_times, threshold=0.7):
    out_survival = model.predict_survival(normed_x, prediction_times)
    fnrs = []
    shot_indices = find_peaks_in_data(normed_t)
    for i, time in enumerate(prediction_times):
        tm_prediction_per_shot = []
        # 1 means correct TM prediction, 0 means unpredicted TM, -1 means no TM in shot
        # a TM is predicted when the survival prediction is 0 at any point in the shot. Check if better results when TM is consecutive 0s
        for j, shot_index in enumerate(shot_indices):
            survival_prediction = out_survival[:,i]
            survival_prediction = (survival_prediction < threshold).astype(int)
            tm = (0 in survival_prediction[shot_indices[j]:shot_indices[j+1]])
            if normed_e[shot_index] == 1 and tm:
                tm_prediction_per_shot.append(1)
            elif normed_e[shot_index] == 1 and not tm:
                tm_prediction_per_shot.append(0)
            else:
                tm_prediction_per_shot.append(-1)
        fnr = tm_prediction_per_shot.count(0) / (tm_prediction_per_shot.count(1) + tm_prediction_per_shot.count(0))
        fnrs.append(fnr)
    auc = np.trapz(fnrs, prediction_times)
    return auc, fnrs, prediction_times

In [86]:
threshold = 0.7
fpr_auc, fprs, prediction_times = fpr_auc(models[0][3], x_test_df, t_test, e_test, prediction_times, threshold=threshold)
fnr_auc, fnrs, prediction_times = fnr_auc(models[0][3], x_test_df, t_test, e_test, prediction_times, threshold=threshold)

/home/hf8585/TMPredictor/survival_tm/auton-survival/auton_survival/estimators.py:206: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return survival_predictions.sort_index(axis=0).interpolate().interpolate(method='bfill').T[times].values
/home/hf8585/TMPredictor/survival_tm/auton-survival/auton_survival/estimators.py:206: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return survival_predictions.sort_index(axis=0).interpolate().interpolate(method='bfill').T[times].values


IndexError: list index out of range